In [16]:
from tardis_dev import datasets, get_exchange_details
import logging
import nest_asyncio
nest_asyncio.apply()
# comment out to disable debug logs
logging.basicConfig(level=logging.DEBUG)

# function used by default if not provided via options
def default_file_name(exchange, data_type, date, symbol, format):
    return f"{exchange}_{data_type}_{date.strftime('%Y-%m-%d')}_{symbol}.{format}.gz"


# customized get filename function - saves data in nested directory structure
def file_name_nested(exchange, data_type, date, symbol, format):
    return f"{exchange}/{data_type}/{date.strftime('%Y-%m-%d')}_{symbol}.{format}.gz"


# returns data available at https://api.tardis.dev/v1/exchanges/deribit
# deribit_details = get_exchange_details("bitget")
# print(deribit_details)

datasets.download(
    # one of https://api.tardis.dev/v1/exchanges with supportsDatasets:true - use 'id' value
    exchange="bitget-futures",
    # accepted data types - 'datasets.symbols[].dataTypes' field in https://api.tardis.dev/v1/exchanges/deribit,
    # or get those values from 'deribit_details["datasets"]["symbols][]["dataTypes"] dict above
    data_types=[ "trades", "quotes",  "book_snapshot_25", "book_snapshot_5"],
    # change date ranges as needed to fetch full month or year for example
    from_date="2025-07-01",
    # to date is non inclusive
    to_date="2025-07-02",
    # accepted values: 'datasets.symbols[].id' field in https://api.tardis.dev/v1/exchanges/deribit
    symbols=["BTCUSDT"],
    # (optional) your API key to get access to non sample data as well
    api_key="TD.28DTwLrn3f988WRM.w3hZLmc9mf2kbFO.OrGHMJWJZldog5f.Qob9tqB1fLEbb9l.0-x0-FrbU6jD9Dz.n2vg",
    # (optional) path where data will be downloaded into, default dir is './datasets'
    # download_dir="./datasets",
    # (optional) - one can customize downloaded file name/path (flat dir strucure, or nested etc) - by default function 'default_file_name' is used
    # get_filename=default_file_name,
    # (optional) file_name_nested will download data to nested directory structure (split by exchange and data type)
    # get_filename=file_name_nested,
)

DEBUG:tardis_dev.datasets.download:download started for bitget-futures trades BTCUSDT from 2025-07-01 to 2025-07-02
DEBUG:tardis_dev.datasets.download:download finished for bitget-futures trades BTCUSDT from 2025-07-01 to 2025-07-02, total time: 5.10207462310791 seconds
DEBUG:tardis_dev.datasets.download:download started for bitget-futures quotes BTCUSDT from 2025-07-01 to 2025-07-02
DEBUG:tardis_dev.datasets.download:download finished for bitget-futures quotes BTCUSDT from 2025-07-01 to 2025-07-02, total time: 2.6983237266540527 seconds
DEBUG:tardis_dev.datasets.download:download started for bitget-futures book_snapshot_25 BTCUSDT from 2025-07-01 to 2025-07-02
DEBUG:tardis_dev.datasets.download:download finished for bitget-futures book_snapshot_25 BTCUSDT from 2025-07-01 to 2025-07-02, total time: 4.398570775985718 seconds
DEBUG:tardis_dev.datasets.download:download started for bitget-futures book_snapshot_5 BTCUSDT from 2025-07-01 to 2025-07-02
DEBUG:tardis_dev.datasets.download:down

In [21]:
import pandas as pd
data_binance_quotes = pd.read_csv('../datasets/binance-futures_quotes_2025-02-01_BTCUSDT.csv')
data_binance_trade = pd.read_csv('../datasets/binance-futures_trades_2025-02-01_BTCUSDT.csv')
# data_binance_trade.isnull().sum()
# data_binance_trade


In [22]:
data_binance_trade

,exchange,symbol,timestamp,local_timestamp,id,side,price,amount
0,binance-futures,BTCUSDT,1738368005344000,1738368005346781,5920536441,buy,102379.8,0.050
1,binance-futures,BTCUSDT,1738368005349000,1738368005351827,5920536442,sell,102379.7,0.008
2,binance-futures,BTCUSDT,1738368005378000,1738368005380764,5920536443,sell,102379.7,0.004
3,binance-futures,BTCUSDT,1738368005380000,1738368005382801,5920536444,buy,102379.8,0.001
4,binance-futures,BTCUSDT,1738368005382000,1738368005384911,5920536445,buy,102379.8,0.001
...,...,...,...,...,...,...,...,...
2078442,binance-futures,BTCUSDT,1738454398515000,1738454398517190,5922615403,buy,100603.6,0.136
2078443,binance-futures,BTCUSDT,1738454398596000,1738454398599113,5922615404,buy,100603.6,0.001
2078444,binance-futures,BTCUSDT,1738454398665000,1738454398667869,5922615405,sell,100603.5,0.003
2078445,binance-futures,BTCUSDT,1738454398672000,1738454398675349,5922615406,sell,100603.5,0.005


In [2]:
data_binance_trade = pd.read_csv('../datasets/binance_trades_2025-02-01_BTCUSDT.csv')
data_binance_snap25 = pd.read_csv('../datasets/binance_book_snapshot_25_2025-02-01_BTCUSDT.csv')

In [57]:
import pandas as pd
import numpy as np 

class Orderbook:
    def __init__(self, symbol):
        self.symbol = symbol
        self.ask = pd.DataFrame(columns=['price', 'size'])
        self.bid = pd.DataFrame(columns=['price', 'size'])
    def add_ask(self, ask_price, ask_size):
        if((abs(self.ask['price'] - ask_price) < 1e-6).any()):
            self.ask.loc[abs(self.ask['price'] - ask_price) < 1e-6, 'size'] += ask_size
        else:
            # self.ask.loc[abs(self.ask['price'] - ask_price) < 1e-6] = pd.DataFrame([ask_price, ask_size])
            self.ask.loc[len(self.ask)] = [ask_price, ask_size]
        self.ask.sort_values(by='price', ascending=False, inplace=True)
    def add_bid(self, bid_price, bid_size):
        if((abs(self.bid['price'] - bid_price) < 1e-6).any()):
            self.bid.loc[abs(self.bid['price'] - bid_price) < 1e-6, 'size'] += bid_size
        else:
            # self.bid.loc[abs(self.bid['price'] - bid_price) < 1e-6] = pd.DataFrame([bid_price, bid_size])
            self.bid.loc[len(self.bid)] = [bid_price, bid_size]
        self.bid.sort_values(by='price', ascending=False, inplace=True)
    def del_ask(self, ask_price):
        if((abs(self.ask['price'] - ask_price) < 1e-6).any()):
            self.ask= self.ask[abs(self.ask['price'] - ask_price) > 1e-6]
        else:
            pass
    def del_bid(self, bid_price):
        if((abs(self.bid['price'] - bid_price) < 1e-6).any()):
            self.bid = self.bid[abs(self.bid['price'] - bid_price) > 1e-6]
        else:
            pass
class MarketMaker:
    def __init__(self, symbol, initial_capital=100000, spread_percent=0.001, max_inventory=0.5, order_size=0.0001):
        self.symbol = symbol
        self.inventory = 0
        self.initial_capital = initial_capital # 初始资金
        self.capital = initial_capital #USDT数量
        self.capital_use = initial_capital # 未用资金
        self.inventory_use = 0  # 未用库存
        self.total = initial_capital #总资产
        self.spread_percent = spread_percent
        self.max_inventory = max_inventory
        self.order_size = order_size
        self.trade_buy = []
        self.trade_sell = []
        self.pnl = pd.DataFrame(columns=['time', 'pnl'])
        self.pending_order = pd.DataFrame(columns=['timestamp','price', 'size','side']) # 下单队列
        self.pending_del = pd.DataFrame(columns=['timestamp','price', 'size','side'])  # 撤单队列
        
    def calculate_spread(self, price):
        ask_price = round((price * (1 + self.spread_percent)) * 10) / 10 # 交易所报价精度为0.1
        bid_price = round(price * (1 - self.spread_percent) * 10) / 10
        # ask_size = self.order_size
        # bid_price = self.order_size
        return ask_price, bid_price
    def update_inventory(self, price, size): # size为正数时买入，为负数时卖出
        self.inventory += size
        self.capital -= price * size
        # self.inventory_use += size
        # if size < 0:
        #     self.capital_use += price * size
        # else:
        #     self.inventory_use += size
    def update_total(self, price, time):
        self.total = self.capital + self.inventory * price
        self.pnl.loc[len(self.pnl)] = [time, self.total - self.initial_capital]
    def update_trade(self, price, size):
        if size > 0:
            self.trade_buy.append([price, size])
        else:
            self.trade_sell.append([price, -size])
    def handle_order(self, price, size, time):
        if (size > 0): # 买单成交
            self.trade_buy.append([price, size])
            self.update_inventory(price, size)
            self.update_total(price,time)

        elif (size < 0): # 卖单成交
            self.trade_sell.append([price, size])
            self.update_inventory(price, size)
            self.update_total(price,time)
        else: # 未成交
            pass
def main():
    symbol = 'BTCUSDT'
    maker = MarketMaker(symbol, 100000, 0.0001, 0.5, 0.0001)
    order = Orderbook('BTCUSDT')

    index = 0
    # while(index < len(data_binance_trade)):
    while(index < 150000):

        price = data_binance_trade.iloc[index]['price']
        maker.update_total(price,data_binance_trade.iloc[index]['timestamp'])
        print(maker.inventory,maker.inventory_use,maker.total)
        # print(maker.inventory_use)
        # # print(maker.capital)
        # print(maker.total)
        
        # print(price)
        if (data_binance_trade.iloc[index]['side'] == 'sell'):
            size = -data_binance_trade.iloc[index]['amount']
        else:
            size = data_binance_trade.iloc[index]['amount']
        # if(order.ask['price'].any() and order.bid['price'].any()):
        i_index = 0
        for i in range(len(maker.pending_order)):
            if(maker.pending_order.iloc[i]['timestamp'] <= data_binance_trade.iloc[index]['timestamp'] - 20000):    #达到20ms 订单被提交
                if(maker.pending_order.iloc[i]['side'] == 'sell'):
                    order.add_ask(maker.pending_order.iloc[i]['price'], maker.pending_order.iloc[i]['size'])
                    # print(order.ask)
                    print('add_ask')
                else:
                    order.add_bid(maker.pending_order.iloc[i]['price'], maker.pending_order.iloc[i]['size'])
                    # print(order.bid)
                    # print('add_bid')
                # maker.pending_order.drop(i, inplace=True)
                i_index += 1
            else:
                break
        maker.pending_order.drop(maker.pending_order.index[range(i_index)], inplace=True)
        if(order.ask['price'].any() and order.bid['price'].any() and data_binance_trade.iloc[index]['side'] == 'buy'):
            ask_enchange = order.ask[order.ask['price'] < price]
            for i_ask in range(len(ask_enchange)):
                print('sell')
                maker.handle_order(ask_enchange.iloc[i_ask]['price'], -ask_enchange.iloc[i_ask]['size'], data_binance_trade.iloc[index]['timestamp'])
                maker.capital_use += ask_enchange.iloc[i_ask]['price'] * ask_enchange.iloc[i_ask]['size']
                # order.del_ask(ask_enchange.iloc[i_ask]['price'])
            order.ask = order.ask[order.ask['price'] >= price]
        elif(order.ask['price'].any() and order.bid['price'].any() and data_binance_trade.iloc[index]['side'] == 'sell'):
            bid_enchange = order.bid[order.bid['price'] > price]
            for i_bid in range(len(bid_enchange)):
                print('buy')
                maker.handle_order(bid_enchange.iloc[i_bid]['price'], bid_enchange.iloc[i_bid]['size'], data_binance_trade.iloc[index]['timestamp'])
                maker.inventory_use += bid_enchange.iloc[i_bid]['size']
            order.bid = order.bid[order.bid['price'] <= price]

        ask_price, bid_price = maker.calculate_spread(price)
        ask_size = maker.order_size
        bid_size = maker.order_size
        delay = data_binance_trade.iloc[index]['timestamp'] + 20000
        if(data_binance_snap25.loc[data_binance_snap25['timestamp'] <= delay,'timestamp'].any()): #若20 ms后ticker挂单会被直接市价成交,则不进入挂单队列
            snap25_time = data_binance_snap25.loc[data_binance_snap25['timestamp'] <= delay,'timestamp'].idxmax()
            ticker_ask = data_binance_snap25.iloc[snap25_time]['asks[0].price'] #此处还没有挂单队列
            ticker_bid = data_binance_snap25.iloc[snap25_time]['bids[0].price']
            if(ticker_ask < bid_price and maker.capital_use > ticker_ask * bid_size):
                maker.update_inventory(ticker_ask, bid_size)        # 买单成交
                maker.update_total(ticker_ask,data_binance_trade.iloc[index]['timestamp'])
                maker.update_trade(ticker_ask, bid_size)
                maker.capital_use -= ticker_ask * bid_size
                maker.inventory_use += bid_size
                # maker.
            elif(ticker_bid > ask_price and maker.inventory_use > ask_size):
                maker.update_inventory(ticker_bid, -ask_size)        # 卖单成交
                maker.update_total(ticker_bid,data_binance_trade.iloc[index]['timestamp'])
                maker.update_trade(ticker_bid, -ask_size)
                maker.capital_use += ticker_bid * ask_size
                maker.inventory_use -= ask_size

            else:
                if(maker.capital_use >= bid_price * bid_size): # 检查资金量是否满足要求
                    maker.pending_order.loc[len(maker.pending_order)] = [data_binance_trade.iloc[index]['timestamp'], bid_price, bid_size, 'buy']
                    maker.capital_use -= bid_price * bid_size
                else:
                    pass
                if(maker.inventory_use >= ask_size): # 检查库存量是否满足要求
                    maker.pending_order.loc[len(maker.pending_order)] = [data_binance_trade.iloc[index]['timestamp'], ask_price, ask_size, 'sell']
                    maker.inventory_use -= ask_size
                else:
                    pass
        else:
            if(maker.capital_use >= bid_price * bid_size): # 检查资金量是否满足要求
                maker.pending_order.loc[len(maker.pending_order)] = [data_binance_trade.iloc[index]['timestamp'], bid_price, bid_size, 'buy']
                maker.capital_use -= bid_price * bid_size
            else:
                pass
            if(maker.inventory_use >= ask_size): # 检查库存量是否满足要求
                maker.pending_order.loc[len(maker.pending_order)] = [data_binance_trade.iloc[index]['timestamp'], ask_price, ask_size, 'sell']
                maker.inventory_use -= ask_size
            else:
                pass


        index = index + 1

main()


0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0
0
0
100000.0

KeyboardInterrupt: 

In [52]:
order = Orderbook('BTCUSDT')
order.add_ask(10000, 10) 
order.add_ask(10000, 10) 
order.add_ask(10000.2, 11)
order.add_ask(10000.1, 12)
order.add_bid(9000, 10)
order.add_bid(9000.1, 11)
order.add_bid(9000.2, 12)
order.add_bid(8999, 13)


In [53]:
order.ask

,price,size
1,10000.2,11.0
2,10000.1,12.0
0,10000.0,20.0


In [54]:
order.del_ask(10000.1)

In [56]:
order.bid

,price,size
2,9000.2,12.0
1,9000.1,11.0
0,9000.0,10.0
3,8999.0,13.0
